In [87]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [88]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [89]:
tf.keras.backend.clear_session()

In [90]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [91]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,247.134
1,2003-2,0.756708,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,305.708
2,2003-3,0.756879,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,284.168
3,2003-4,0.757050,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,286.594
4,2003-5,0.757222,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,559.950


In [92]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.524203,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.508666,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.874645,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771
188,1.308842,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.890157,0.806353,0.784605,1.140354,-0.757425,0.173158,0.684584
189,1.302363,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.906034,0.844284,0.779552,1.131439,-0.742194,0.162516,0.674397
190,1.295885,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.919415,0.884141,0.774498,1.122523,-0.726963,0.151874,0.664209


In [93]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [94]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.524203,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.508666,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.399834,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.832198,0.050588,1.519517,1.163052,-0.960847,-0.093961,0.627995
158,1.402752,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.824048,0.042723,1.476375,1.168317,-0.968515,-0.074705,0.638908
159,1.405671,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.815727,0.029152,1.433232,1.173583,-0.976183,-0.055448,0.649820
160,1.408589,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.810089,-0.007439,1.390090,1.178848,-0.983851,-0.036192,0.660733


In [95]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
157    338.767
158    419.962
159    387.800
160    391.122
161    409.739
Name: Minas Gerais - Consumo de Cimento (t), Length: 162, dtype: float64

In [96]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [97]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [98]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
126,0.544198,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.135116,0.964210,1.008346,0.811298,0.287865,0.502913,0.863809
127,0.559144,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.150078,0.924638,1.051039,0.826912,0.237130,0.464014,0.848131
128,0.574090,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.162632,0.882104,1.093731,0.842525,0.186395,0.425115,0.832454
129,0.589036,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.166985,0.846537,1.136424,0.858138,0.135660,0.386216,0.816777
130,0.603982,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.145581,0.796933,1.179116,0.873751,0.084924,0.347317,0.801100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.874645,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771
188,1.308842,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.890157,0.806353,0.784605,1.140354,-0.757425,0.173158,0.684584
189,1.302363,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.906034,0.844284,0.779552,1.131439,-0.742194,0.162516,0.674397
190,1.295885,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.919415,0.884141,0.774498,1.122523,-0.726963,0.151874,0.664209


In [99]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [100]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [101]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 10)
#     target,target_val = validation_splitter(train_target, 10)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(180, activation='tanh', 
                             return_sequences=True,
                             kernel_regularizer=tf.keras.regularizers.l1(0.01),
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True,
                             kernel_regularizer=tf.keras.regularizers.l1(0.01)),
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=False),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [102]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [103]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2600928851, 1859697737, 1044811374, 1352467791, 100136943, 3169695251, 1929701926, 1173774545, 4047294686, 3007562141]


Step: 0 ___________________________________________
val_loss: 486.7550048828125
winner_seed: 2600928851


Step: 1 ___________________________________________
val_loss: 486.7563171386719


Step: 2 ___________________________________________
val_loss: 486.75274658203125
winner_seed: 1044811374


Step: 3 ___________________________________________
val_loss: 486.7522888183594
winner_seed: 1352467791


Step: 4 ___________________________________________
val_loss: 486.7537536621094


Step: 5 ___________________________________________
val_loss: 486.7532043457031


Step: 6 ___________________________________________
val_loss: 486.75274658203125


Step: 7 ___________________________________________
val_loss: 486.75286865234375


Step: 8 ___________________________________________
val_loss: 486.75421142578125


Step: 9 ___________________________________________
val_loss: 48

In [104]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 159ms/step - loss: 307224.6875 - val_loss: 163718.0000
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 305213.6250 - val_loss: 162189.3906
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 303530.6875 - val_loss: 159969.8750
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 302286.3750 - val_loss: 159008.5938
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 301284.8438 - val_loss: 158375.7969
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 300491.7500 - val_loss: 157876.8125
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 299864.1875 - val_loss: 157485.2344
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 299367.4375 - val_loss: 157177.0938
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 298977.4375 - val_loss: 156926.7500
Epoch 10/1000

4/4 [==============================] - 0s 10ms/step - loss: 287177.6250 - val_loss: 148416.2031
Epoch 76/10000
4/4 [==============================] - 0s 10ms/step - loss: 287018.0625 - val_loss: 148299.4219
Epoch 77/10000
4/4 [==============================] - 0s 11ms/step - loss: 286857.1250 - val_loss: 148182.9531
Epoch 78/10000
4/4 [==============================] - 0s 11ms/step - loss: 286697.7500 - val_loss: 148066.5000
Epoch 79/10000
4/4 [==============================] - 0s 11ms/step - loss: 286535.5625 - val_loss: 147950.4688
Epoch 80/10000
4/4 [==============================] - 0s 10ms/step - loss: 286376.0938 - val_loss: 147834.2969
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 286216.0312 - val_loss: 147718.2188
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 286056.4688 - val_loss: 147602.1562
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 285896.9688 - val_loss: 147486.0469
Epoch 84/10000
4

4/4 [==============================] - 0s 10ms/step - loss: 275752.7188 - val_loss: 140157.1250
Epoch 149/10000
4/4 [==============================] - 0s 10ms/step - loss: 275598.5938 - val_loss: 140047.0000
Epoch 150/10000
4/4 [==============================] - 0s 11ms/step - loss: 275446.6875 - val_loss: 139936.6562
Epoch 151/10000
4/4 [==============================] - 0s 10ms/step - loss: 275291.8125 - val_loss: 139826.8906
Epoch 152/10000
4/4 [==============================] - 0s 11ms/step - loss: 275140.9062 - val_loss: 139716.5938
Epoch 153/10000
4/4 [==============================] - 0s 11ms/step - loss: 274986.8750 - val_loss: 139606.7500
Epoch 154/10000
4/4 [==============================] - 0s 11ms/step - loss: 274833.2812 - val_loss: 139497.0312
Epoch 155/10000
4/4 [==============================] - 0s 12ms/step - loss: 274682.7188 - val_loss: 139386.6719
Epoch 156/10000
4/4 [==============================] - 0s 12ms/step - loss: 274529.8438 - val_loss: 139276.6562
Epoch 15

4/4 [==============================] - 0s 11ms/step - loss: 264773.2812 - val_loss: 132285.1562
Epoch 222/10000
4/4 [==============================] - 0s 10ms/step - loss: 264626.7812 - val_loss: 132179.5938
Epoch 223/10000
4/4 [==============================] - 0s 11ms/step - loss: 264478.5938 - val_loss: 132074.3594
Epoch 224/10000
4/4 [==============================] - 0s 10ms/step - loss: 264330.8750 - val_loss: 131969.2344
Epoch 225/10000
4/4 [==============================] - 0s 10ms/step - loss: 264183.0938 - val_loss: 131864.0469
Epoch 226/10000
4/4 [==============================] - 0s 10ms/step - loss: 264035.7188 - val_loss: 131758.6562
Epoch 227/10000
4/4 [==============================] - 0s 10ms/step - loss: 263888.3125 - val_loss: 131653.3594
Epoch 228/10000
4/4 [==============================] - 0s 11ms/step - loss: 263740.8438 - val_loss: 131548.0625
Epoch 229/10000
4/4 [==============================] - 0s 10ms/step - loss: 263593.0000 - val_loss: 131442.7500
Epoch 23

4/4 [==============================] - 0s 10ms/step - loss: 254151.6094 - val_loss: 124731.1797
Epoch 295/10000
4/4 [==============================] - 0s 11ms/step - loss: 254007.0156 - val_loss: 124630.2500
Epoch 296/10000
4/4 [==============================] - 0s 11ms/step - loss: 253865.7188 - val_loss: 124528.8828
Epoch 297/10000
4/4 [==============================] - 0s 11ms/step - loss: 253722.1719 - val_loss: 124427.9062
Epoch 298/10000
4/4 [==============================] - 0s 11ms/step - loss: 253579.0625 - val_loss: 124327.1172
Epoch 299/10000
4/4 [==============================] - 0s 11ms/step - loss: 253437.2031 - val_loss: 124226.0859
Epoch 300/10000
4/4 [==============================] - 0s 11ms/step - loss: 253293.1250 - val_loss: 124125.7344
Epoch 301/10000
4/4 [==============================] - 0s 11ms/step - loss: 253152.1406 - val_loss: 124024.8750
Epoch 302/10000
4/4 [==============================] - 0s 10ms/step - loss: 253009.3438 - val_loss: 123923.9922
Epoch 30

...

Epoch 3108/10000
4/4 [==============================] - 0s 8ms/step - loss: 22315.0488 - val_loss: 837.0362
Epoch 3109/10000
4/4 [==============================] - 0s 8ms/step - loss: 22285.5566 - val_loss: 841.4493
Epoch 3110/10000
4/4 [==============================] - 0s 8ms/step - loss: 22256.6113 - val_loss: 845.9329
Epoch 3111/10000
4/4 [==============================] - 0s 9ms/step - loss: 22225.6504 - val_loss: 850.3734
Epoch 3112/10000
4/4 [==============================] - 0s 8ms/step - loss: 22196.9785 - val_loss: 854.9040
Epoch 3113/10000
4/4 [==============================] - 0s 8ms/step - loss: 22166.4395 - val_loss: 859.3915
Epoch 3114/10000
4/4 [==============================] - 0s 8ms/step - loss: 22137.2949 - val_loss: 863.9528
Epoch 3115/10000
4/4 [==============================] - 0s 8ms/step - loss: 22107.7051 - val_loss: 868.5293
Epoch 3116/10000
4/4 [==============================] - 0s 8ms/step - loss: 22078.5000 - val_loss: 873.1265
Epoch 3117/10000
4/4 [======

4/4 [==============================] - 0s 9ms/step - loss: 20181.2324 - val_loss: 1239.7040
Epoch 3184/10000
4/4 [==============================] - 0s 9ms/step - loss: 20153.6387 - val_loss: 1245.9658
Epoch 3185/10000
4/4 [==============================] - 0s 9ms/step - loss: 20126.9121 - val_loss: 1252.3345
Epoch 3186/10000
4/4 [==============================] - 0s 9ms/step - loss: 20100.7930 - val_loss: 1258.8551
Epoch 3187/10000
4/4 [==============================] - 0s 9ms/step - loss: 20072.6660 - val_loss: 1265.3138
Epoch 3188/10000
4/4 [==============================] - 0s 8ms/step - loss: 20045.2305 - val_loss: 1271.7408
Epoch 3189/10000
4/4 [==============================] - 0s 9ms/step - loss: 20017.7402 - val_loss: 1278.1627
Epoch 3190/10000
4/4 [==============================] - 0s 9ms/step - loss: 19991.4141 - val_loss: 1284.7114
Epoch 3191/10000
4/4 [==============================] - 0s 9ms/step - loss: 19964.1699 - val_loss: 1291.2542
Epoch 3192/10000
4/4 [==============

4/4 [==============================] - 0s 9ms/step - loss: 18232.6113 - val_loss: 1783.8710
Epoch 3259/10000
4/4 [==============================] - 0s 9ms/step - loss: 18207.2031 - val_loss: 1792.0742
Epoch 3260/10000
4/4 [==============================] - 0s 9ms/step - loss: 18183.2363 - val_loss: 1800.3436
Epoch 3261/10000
4/4 [==============================] - 0s 9ms/step - loss: 18157.4961 - val_loss: 1808.4636
Epoch 3262/10000
4/4 [==============================] - 0s 9ms/step - loss: 18132.7969 - val_loss: 1816.6086
Epoch 3263/10000
4/4 [==============================] - 0s 9ms/step - loss: 18108.3691 - val_loss: 1824.8523
Epoch 3264/10000
4/4 [==============================] - 0s 9ms/step - loss: 18083.9043 - val_loss: 1833.1818
Epoch 3265/10000
4/4 [==============================] - 0s 8ms/step - loss: 18058.9434 - val_loss: 1841.5083
Epoch 3266/10000
4/4 [==============================] - 0s 8ms/step - loss: 18034.9863 - val_loss: 1849.9954
Epoch 3267/10000
4/4 [==============

4/4 [==============================] - 0s 9ms/step - loss: 16457.6367 - val_loss: 2462.5261
Epoch 3334/10000
4/4 [==============================] - 0s 9ms/step - loss: 16435.1797 - val_loss: 2472.4355
Epoch 3335/10000
4/4 [==============================] - 0s 9ms/step - loss: 16412.5723 - val_loss: 2482.4219
Epoch 3336/10000
4/4 [==============================] - 0s 9ms/step - loss: 16390.6504 - val_loss: 2492.5308
Epoch 3337/10000
4/4 [==============================] - 0s 8ms/step - loss: 16367.6055 - val_loss: 2502.5479
Epoch 3338/10000
4/4 [==============================] - 0s 8ms/step - loss: 16345.7031 - val_loss: 2512.6450
Epoch 3339/10000
4/4 [==============================] - 0s 9ms/step - loss: 16323.2236 - val_loss: 2522.6294
Epoch 3340/10000
4/4 [==============================] - 0s 8ms/step - loss: 16300.9570 - val_loss: 2532.5669
Epoch 3341/10000
4/4 [==============================] - 0s 9ms/step - loss: 16278.8115 - val_loss: 2542.4792
Epoch 3342/10000
4/4 [==============

4/4 [==============================] - 0s 9ms/step - loss: 14856.0547 - val_loss: 3266.7881
Epoch 3409/10000
4/4 [==============================] - 0s 9ms/step - loss: 14835.6895 - val_loss: 3278.2075
Epoch 3410/10000
4/4 [==============================] - 0s 9ms/step - loss: 14815.5430 - val_loss: 3289.6323
Epoch 3411/10000
4/4 [==============================] - 0s 8ms/step - loss: 14795.2783 - val_loss: 3301.0503
Epoch 3412/10000
4/4 [==============================] - 0s 9ms/step - loss: 14776.1895 - val_loss: 3312.7061
Epoch 3413/10000
4/4 [==============================] - 0s 9ms/step - loss: 14755.2217 - val_loss: 3324.0669
Epoch 3414/10000
4/4 [==============================] - 0s 9ms/step - loss: 14735.8350 - val_loss: 3335.7112
Epoch 3415/10000
4/4 [==============================] - 0s 9ms/step - loss: 14715.9014 - val_loss: 3347.4944
Epoch 3416/10000
4/4 [==============================] - 0s 9ms/step - loss: 14695.0391 - val_loss: 3359.1343
Epoch 3417/10000
4/4 [==============

In [105]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [106]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572,403.861572
Target,413.46,497.019,476.478,421.395,413.641,338.033,431.028,305.105,402.406,419.419,336.054,568.395,541.488,537.851,526.265,559.293,466.646,416.049,501.489,460.699,445.583,509.271,532.602,500.905,588.495,589.349,543.982,614.074,557.419,412.582
Error,9.598419,93.15744,72.616425,17.533417,9.779419,65.828583,27.166443,98.756561,1.455566,15.557434,67.807587,164.533447,137.626404,133.989441,122.403442,155.431458,62.784424,12.187439,97.627441,56.837433,41.721436,105.409424,128.740417,97.043427,184.633423,185.487427,140.120422,210.212402,153.557434,8.720428


In [107]:
display(mae)
display(mape)

89.27748

0.17661437

In [108]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [109]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[4846.339]] - Target[5022.433000000001]| =  Error: [[176.09424]]; MAPE:[[0.03506154]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[4846.339]] - Target[5998.141]| =  Error: [[1151.8022]]; MAPE:[[0.19202654]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[2423.1694]] - Target[3305.901]| =  Error: [[882.73145]]; MAPE:[[0.26701692]]


[array([[176.09424]], dtype=float32),
 array([[1151.8022]], dtype=float32),
 array([[882.73145]], dtype=float32)]

736.876

0.16470166